In [21]:
import os
import scipy.io as io
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import sys
import math
from netCDF4 import Dataset
import os, calendar, sys, fnmatch, datetime

# Load Huang et al. 2023 Ellipsoidal dust optical properties files

In [22]:
# ===== Aerosol (e.g., dust) optical properties (i.e., Qext, SSA, g) =====
# ***Note: The order of 14 bands is listed in TABLE 1 in RRTM_SW instruction (It is not monotonically increase or decrease)
# ***Note: The order of spectral optical properties should be consistent with the order in TABLE 1

def process_optical_properties(wavelength_band, txt_files):
    wavelength = wavelength_range[wavelength_band]
    #print(txt_files[wavelength_band])
    opt_prop_sw_original = np.loadtxt(opt_dir + '{}'.format(txt_files[wavelength_band]))
    #print(opt_prop_sw_original.shape)

    wl = np.array(opt_prop_sw_original[:, 0][:])
    dust_d = np.array(opt_prop_sw_original[:, 1][:])
    size_parameter = np.pi * dust_d / wl #f'size_parameter_{wavelength:.4f}': size_parameter,
    n_RI = np.array(opt_prop_sw_original[:, 2][:])
    k_RI = np.array(opt_prop_sw_original[:, 3][:])
    
    qe_dust_sw = opt_prop_sw_original[:, 4]
    ssa_dust_sw = opt_prop_sw_original[:, 5]
    g_dust_sw = opt_prop_sw_original[:, 6]

    # Create variable names dynamically
    variables = {
        f'wl_{wavelength:.4f}': wl,
        f'dust_d_{wavelength:.4f}': dust_d,
        f'size_parameter_{wavelength:.4f}': size_parameter,
        f'n_RI_{wavelength:.4f}': n_RI,
        f'k_RI_{wavelength:.4f}': k_RI,
        f'qe_dust_sw_{wavelength:.4f}': qe_dust_sw,
        f'ssa_dust_sw_{wavelength:.4f}': ssa_dust_sw,
        f'g_dust_sw_{wavelength:.4f}': g_dust_sw,
    }
    return np.array((wl,dust_d,size_parameter,n_RI,k_RI,qe_dust_sw,ssa_dust_sw,g_dust_sw)) #variables


In [23]:
# SW Wavelength range and files
opt_dir = 'SW_Huang_Dust_Optics/'
wavelength_range = [0.35, 0.55, 0.75, 0.95] #, 2, 4, 6, 8, 10, 12, 14, 16, 35]
Huang_2023_dust_opt_prop_txt_files = ['shapein_TAMU_0.3500.txt', 'shapein_TAMU_0.5500.txt', 'shapein_TAMU_0.7500.txt', 'shapein_TAMU_0.9500.txt'] # 'shapein_TAMU_2.0000.txt', 'shapein_TAMU_4.0000.txt', 'shapein_TAMU_6.0000.txt', 'shapein_TAMU_8.0000.txt', 'shapein_TAMU_10.0000.txt', 'shapein_TAMU_12.0000.txt', 'shapein_TAMU_14.0000.txt', 'shapein_TAMU_16.0000.txt', 'shapein_TAMU_35.0000.txt']

# Array to store results
opt_prop_sw_array = np.empty(len(wavelength_range), dtype=object)

# Process optical properties for each wavelength band
for i in range(len(wavelength_range)):
    opt_prop_sw = process_optical_properties(i, Huang_2023_dust_opt_prop_txt_files)
    opt_prop_sw_array[i] = opt_prop_sw

In [24]:
print(opt_prop_sw_array[0].shape)

(8, 14400)


In [25]:
opt_prop_sw_array[0][5].shape

(14400,)

## Create Yue Size parameter, real RI, and imaginary RI matrix

create a file of the three dimensional grid, those three dimensions being size parameter, real RI, and imaginary RI. The second and third dimensions here are easier and would be those values relevant to your problem, so I'd suggest (1.50, 1.52, 1.53, 1.56) and (0.0005, 0.001, 0.002, 0.003). The first dimension is pretty annoying because there will be so many values since the data is finely spaced in D (300 values per wavelength) and you will have four different SW wavelengths to use (350, 550, 750, and 950 nm; don't use the other ones because the RI there is for the LW). So I'd suggest only using every 10th value to keep the size of the matrix reasonable (120 size parameter values). Once you've created this three-dimensional grid (120x4x4), 

In [47]:
Yue_real_RI_range = np.array([1.50, 1.52, 1.53, 1.56])
Yue_imag_RI_range = np.array([0.0005, 0.001, 0.002, 0.003])
size_param_array_length = int(len(opt_prop_sw_array[0][0])/10)
print(size_param_array_length)
Yue_size_parameter_range = np.zeros(size_param_array_length, dtype=object)
for i in range(0,len(wavelength_range),10):
    Yue_size_parameter_range[i] = opt_prop_sw_array[i][2]
print(Yue_size_parameter_range.shape,Yue_size_parameter_range)

Yue_size_parameter_and_Ri_matrix = np.array((Yue_size_parameter_range, Yue_real_RI_range, Yue_imag_RI_range), dtype=object)
print(Yue_size_parameter_and_Ri_matrix.shape,Yue_size_parameter_and_Ri_matrix)

io.savemat("Yue_size_parameter_and_Ri_matrix.mat", {"array": Yue_size_parameter_and_Ri_matrix})

1440
(1440,) [array([  0.8975979 ,   0.91824265,   0.939785  , ..., 606.3462317 ,
        617.23499184, 628.31853072])                                 0 0
 ... 0 0 0]
(3,) [array([array([  0.8975979 ,   0.91824265,   0.939785  , ..., 606.3462317 ,
               617.23499184, 628.31853072])                                ,
        0, 0, ..., 0, 0, 0], dtype=object)
 array([1.5 , 1.52, 1.53, 1.56]) array([0.0005, 0.001 , 0.002 , 0.003 ])]


In [27]:
Yue_size_parameter_and_Ri_matrix[0]

array([array([  0.8975979 ,   0.91824265,   0.939785  , ..., 606.3462317 ,
              617.23499184, 628.31853072])                                ,
       0, 0, ..., 0, 0, 0], dtype=object)

## Create Yue Q_e matrix

 you need to create a matrix with the same dimensions as your three-dimensional grid that contains the values of Q_ext. You do this by creating a matrix with NaN values with those dimensions and then reading in all Yue's data (at the four wavelengths, using only every tenth data point) and writing each value to this new matrix at the corresponding coordinate. Once you're done, you'll end with a matrix that has a lot of values filled in, but ~75% of the values will be NaNs. 

# Calculate DustCOMM mean geometric diamater of each bin

In [8]:
#----------
# using DustCOMM 2021 papers b) https://dustcomm.atmos.ucla.edu/
# specifically https://dustcomm.atmos.ucla.edu/data/K21b/ dataset DustCOMM_source_region_DAOD_seas_bin_abs.nc
# assumes aspherical dust shape, PM20 dust
#----------

daod_Dustcomm_PM20_Dataset = Dataset('DustCOMM_source_region_DAOD_annual_bin_abs.nc')
print(daod_Dustcomm_PM20_Dataset) #[season,source,diameter,lat,lon,bin]

lat_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lat'][:])
lon_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lon'][:])
mean_Daod_Dustcomm_20PM = np.array(daod_Dustcomm_PM20_Dataset.variables['Mean'][:])
#dustcomm_seas = np.array(daod_Dustcomm_PM20_Dataset.variables['season'][:])
dustcomm_Sources_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['source'][:]) 
dustcomm_bin_lower = np.array(daod_Dustcomm_PM20_Dataset.variables['bin_D_lower'][:])
dustcomm_bin_upper = np.array(daod_Dustcomm_PM20_Dataset.variables['bin_D_upper'][:])

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): source(9), diameter(6), lat(96), lon(144), bin(6)
    variables(dimensions): float64 source(source), float64 bin_D_lower(diameter), float64 bin_D_upper(diameter), float64 lat(lat), float64 lon(lon), float64 Median(lat, lon, bin, source), float64 Mean(lat, lon, bin, source), float64 Neg1sigma(lat, lon, bin, source), float64 Pos1sigman(lat, lon, bin, source), float64 Neg2sigma(lat, lon, bin, source), float64 Pos2sigma(lat, lon, bin, source)
    groups: 


In [9]:
#----------
# using DustCOMM 2021 papers b) https://dustcomm.atmos.ucla.edu/
# specifically https://dustcomm.atmos.ucla.edu/data/K21b/ dataset DustCOMM_source_region_DAOD_seas_bin_abs.nc
# assumes aspherical dust shape, PM20 dust
#----------

loading_Dustcomm_PM20_Dataset = Dataset('DustCOMM_source_region_dust_loading_annual_bin_abs.nc')
print(loading_Dustcomm_PM20_Dataset) #[season,source,diameter,lat,lon,bin]

loading_lat_Dustcomm_PM20 = np.array(loading_Dustcomm_PM20_Dataset.variables['lat'][:])
loading_lon_Dustcomm_PM20 = np.array(loading_Dustcomm_PM20_Dataset.variables['lon'][:])
mean_loading_Dustcomm_20PM = np.array(loading_Dustcomm_PM20_Dataset.variables['Mean'][:])
#dustcomm_seas = np.array(daod_Dustcomm_PM20_Dataset.variables['season'][:])
loading_dustcomm_Sources_PM20 = np.array(loading_Dustcomm_PM20_Dataset.variables['source'][:]) 
loading_dustcomm_bin_lower = np.array(loading_Dustcomm_PM20_Dataset.variables['bin_D_lower'][:])
loading_dustcomm_bin_upper = np.array(loading_Dustcomm_PM20_Dataset.variables['bin_D_upper'][:])

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): source(9), diameter(6), lat(96), lon(144), bin(6)
    variables(dimensions): float64 source(source), float64 bin_D_lower(diameter), float64 bin_D_upper(diameter), float64 lat(lat), float64 lon(lon), float64 Median(lat, lon, bin, source), float64 Mean(lat, lon, bin, source), float64 Neg1sigma(lat, lon, bin, source), float64 Pos1sigman(lat, lon, bin, source), float64 Neg2sigma(lat, lon, bin, source), float64 Pos2sigma(lat, lon, bin, source)
    groups: 


In [10]:
print(dustcomm_bin_lower)
print(dustcomm_bin_upper)

[ 0.2  0.5  1.   2.5  5.  10. ]
[ 0.5  1.   2.5  5.  10.  20. ]


In [11]:
mean_geometric_diameter_bin = np.sqrt(dustcomm_bin_lower*dustcomm_bin_upper)

In [12]:
print(mean_geometric_diameter_bin)

[ 0.31622777  0.70710678  1.58113883  3.53553391  7.07106781 14.14213562]


# Interpolate  Huang et al. 2023 size parameter and dust optical properties that best match Adebiyi RI's, and DustCOMM size parameter of each bin

use DustCOMM Saharan and Sahelian dust partition in a grid box, in the DAOD. sum the two sahara sources, to calculate my own spatially resolved RI (Later)

match_d_opt_prop_sw = []
match_n_RI_opt_prop_sw = []
match_k_RI_opt_prop_sw = []
n_RI_index = []
k_RI_index = []North African dust has a mean imaginary refractive index at 550 nm wavelength of 0.0012 (one standard error range of 0.0009–0.0016
and a real dust refractive index, nr = 1.51 ± 0.03 (which is the same for Sahara and Sahel) (Di Biagio et al. 2019)
We assume that RI is constant across particle size (Adebiyi et al. 2023, Obiso et al. 2023)

In [79]:
# 14 RRTM SW bands - average wavelength of each band (in RRTM order)
RRTM_SW_bands = [3.4615, 2.7885, 2.3255, 2.0465, 1.784, 1.4625, 1.2705, 1.01, 0.7015, 0.5335, 0.3935, 0.304, 0.2315, 8.0205]
RRTM_SW_bands_sorted = np.sort(RRTM_SW_bands)

In [120]:
#grid_box = [0,18] #0W, 18N
North_African_dust_adebiyi_k = np.full(len(mean_geometric_diameter_bin),0.0012)
North_African_dust_adebiyi_nr = np.full(len(mean_geometric_diameter_bin),1.51)

In [121]:
print(North_African_dust_adebiyi_nr)

[1.51 1.51 1.51 1.51 1.51 1.51]


Meeting Notes 1/4/24:

Look at how solar zenith angle variation effects "DRE" (difference between the two fluxes)
Send Jasper bulk dust optical properties by end of day tomorrow (plus text file)
write script to run SW RRTM for different solar zenith angles

In [127]:
size_parameter_geometric_diameter_bin = np.zeros(len(opt_prop_sw_array)+1, dtype=object)

for irrtm in range(len(opt_prop_sw_array)+1):
    size_parameter_geometric_diameter_bin[irrtm] = np.pi*mean_geometric_diameter_bin/RRTM_SW_bands[irrtm] #not sorted, in RRTM order
print(size_parameter_geometric_diameter_bin)

[array([ 0.28700241,  0.64175689,  1.43501203,  3.20878444,  6.41756888,
        12.83513777])
 array([ 0.35626998,  0.79664388,  1.78134988,  3.98321942,  7.96643883,
        15.93287767])
 array([ 0.42720225,  0.95525327,  2.13601124,  4.77626633,  9.55253266,
        19.10506531])
 array([ 0.48544287,  1.08548325,  2.42721433,  5.42741625, 10.85483249,
        21.70966498])
 array([ 0.55687154,  1.24520262,  2.7843577 ,  6.22601309, 12.45202617,
        24.90405234])
 array([ 0.67928809,  1.51893434,  3.39644043,  7.59467169, 15.18934338,
        30.37868676])
 array([ 0.78194319,  1.74847813,  3.90971596,  8.74239067, 17.48478134,
        34.96956268])
 array([ 0.9836226 ,  2.199447  ,  4.918113  , 10.997235  , 21.99446999,
        43.98893998])
 array([ 1.4161922 ,  3.16670202,  7.08096099, 15.83351011, 31.66702023,
        63.33404046])
 array([ 1.86215338,  4.16390154,  9.31076688, 20.81950768, 41.63901535,
        83.27803071])
 array([  2.524673  ,   5.64534046,  12.62336501, 

## 1/22/24 Interpolate for each dust optical property

In [123]:
from scipy.interpolate import RegularGridInterpolator

In [124]:
opt_prop_sw_array[0][2]

array([  0.8975979 ,   0.91824265,   0.939785  , ..., 606.3462317 ,
       617.23499184, 628.31853072])

In [125]:
size_parameter_geometric_diameter_bin[6]

array([ 0.78194319,  1.74847813,  3.90971596,  8.74239067, 17.48478134,
       34.96956268])

In [137]:
from scipy.interpolate import griddata 

def interpolate_bin_size_param_Qe(bin_data, iwl, irrtm):
    x = bin_data[iwl][2]  # size parameter
    y = bin_data[iwl][3]  # real RI
    z = bin_data[iwl][4]  # imaginary RI
    Q_e = bin_data[iwl][5]

    # Check for nan values and handle them (fill with a specific value, e.g., 0)
    x = np.nan_to_num(x, nan=0.0)
    y = np.nan_to_num(y, nan=0.0)
    z = np.nan_to_num(z, nan=0.0)
    Q_e = np.nan_to_num(Q_e, nan=0.0)

    # Define points where you want to interpolate
    x_interp = size_parameter_geometric_diameter_bin[irrtm]
    y_interp = North_African_dust_adebiyi_nr
    z_interp = North_African_dust_adebiyi_k

    # Create a meshgrid from interpolation points
    x_interp, y_interp, z_interp = np.meshgrid(x_interp, y_interp, z_interp, indexing='ij')

    # Combine the coordinates into a 2D array of points
    points = np.column_stack((x_interp.flatten(), y_interp.flatten(), z_interp.flatten()))

    # Interpolate the values using griddata and handle nan values with fill_value
    fill_value = np.nan  # specify the fill_value for nan
    result = griddata((x.flatten(), y.flatten(), z.flatten()), Q_e.flatten(), points, method='linear', fill_value=fill_value)

    # Reshape the result to match the shape of the input meshgrid
    result = result.reshape(x_interp.shape)

    # Print the interpolated values
    print("Interpolated Qe Values:", np.array((result[0][0][0],result[1][0][0],result[2][0][0],result[3][0][0],result[4][0][0],result[5][0][0])))
    return np.array((result[0][0][0],result[1][0][0],result[2][0][0],result[3][0][0],result[4][0][0],result[5][0][0]))

arrayy = np.zeros((len(opt_prop_sw_array)+1, len(opt_prop_sw_array)), dtype=object)
# Example usage
for iwl in range(len(opt_prop_sw_array)):
    for irrtm in range(len(size_parameter_geometric_diameter_bin)):
        arrayy[irrtm,iwl] = interpolate_bin_size_param_Qe(opt_prop_sw_array, iwl,irrtm)
#interpolate_bin_size_param_Qe(opt_prop_sw_array, 0,0)

Interpolated Qe Values: [       nan        nan 1.0453977  4.33971603 5.17817266 3.65934204]
Interpolated Qe Values: [       nan        nan 1.76099469 5.07836835 4.48590485 3.69420447]
Interpolated Qe Values: [       nan 0.34371527 2.52618736 5.34368378 3.9378878  3.60314327]
Interpolated Qe Values: [       nan 0.50243592 3.12630312 5.36334701 3.72727989 3.49771888]
Interpolated Qe Values: [       nan 0.72649966 3.7296873  5.21001789 3.66724195 3.4198482 ]
Interpolated Qe Values: [       nan 1.20994309 4.57627027 4.64087842 3.72972444 3.33047059]
Interpolated Qe Values: [       nan 1.69588056 4.98978869 4.18339518 3.6697722  3.28100217]
Interpolated Qe Values: [0.36361828 2.67060382 5.43334262 3.70634405 3.49556094 3.18085632]
Interpolated Qe Values: [1.01335823 4.31551542 4.86343423 3.69034112 3.32740056 3.05778172]
Interpolated Qe Values: [1.91902789 5.19239671 3.98291121 3.55727662 3.18251315 3.01462116]
Interpolated Qe Values: [3.30838527 5.36369982 3.66248712 3.35973818 3.09783609 

KeyboardInterrupt: 

# Calculate bulk dust Optical properties for (0W, 18N)

Reorder dust optical properties so it is in order (Q_e, ssa, g)

In [ ]:
# Calculate and print for each bin
all_bins_opt_prop_array = []

# Convert the list to a NumPy array
all_bins_opt_prop_array = np.array((bin1_opt_prop_interp_size_param_y, bin2_opt_prop_interp_size_param_y, bin3_opt_prop_interp_size_param_y, bin4_opt_prop_interp_size_param_y, bin5_opt_prop_interp_size_param_y, bin6_opt_prop_interp_size_param_y))
print(all_bins_opt_prop_array)

In [ ]:
print(loading_lat_Dustcomm_PM20.shape)
print(loading_lon_Dustcomm_PM20.shape)
print(mean_loading_Dustcomm_20PM.shape)
print(mean_loading_Dustcomm_20PM[0,0,1,2])
#dimensions(sizes): source(9), diameter(6), lat(96), lon(144), bin(6)
index_0W = []
index_18N = []

lat_0W = 0
lon_18N = 18

for ilat in range(0,len(loading_lat_Dustcomm_PM20)):
    if np.round(loading_lat_Dustcomm_PM20[ilat]) -1 == lat_0W:
        print(loading_lat_Dustcomm_PM20[ilat])
        index_0W.append(ilat)

for ilon in range(0,len(loading_lon_Dustcomm_PM20)):
    if np.round(loading_lon_Dustcomm_PM20[ilon]) == lon_18N: 
        print(loading_lon_Dustcomm_PM20[ilon])
        index_18N.append(ilon)
print(index_0W,index_18N)

In [ ]:
total_North_African_loading = mean_loading_Dustcomm_20PM[:,:,:,0] + mean_loading_Dustcomm_20PM[:,:,:,1] + mean_loading_Dustcomm_20PM[:,:,:,2]
Sarahan_laoding = mean_loading_Dustcomm_20PM[:,:,:,0] + mean_loading_Dustcomm_20PM[:,:,:,1]
Sahelian_loading = mean_loading_Dustcomm_20PM[:,:,:,2]
print(total_North_African_loading.shape)

In [ ]:
#ρ_d=(2.5±0.2)×10^3 kg m-3 is the globally representative density of dust aerosols (Fratini et al., 2007; Reid et al., 2008; Kaaden et al., 2009; Sow et al., 2009)
ρ_d = 2.5e3
delta_m_Db = total_North_African_loading[48,78,:]
print('delta_m_Db',delta_m_Db)

Q_e_bulk_0W_18N_bin_num = np.zeros(6)
Q_e_bulk_0W_18N_bin_den = np.zeros(6)
SSA_bulk_0W_18N_bin_num = np.zeros(6)
SSA_bulk_0W_18N_bin_den = np.zeros(6)
Q_scat = np.zeros(6)
g_bulk_0W_18N_bin_num = np.zeros(6)
g_bulk_0W_18N_bin_den = np.zeros(6)

#all_bins_opt_prop_array[i][0] = Q_e of bin i
#all_bins_opt_prop_array[i][1] = SSA of bin i
#all_bins_opt_prop_array[i][2] = g of bin i

for i in range(0,len(all_bins_opt_prop_array)):
    Q_e_bulk_0W_18N_bin_num[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*all_bins_opt_prop_array[i][0])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    Q_e_bulk_0W_18N_bin_den[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2))/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    
    SSA_bulk_0W_18N_bin_num[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*all_bins_opt_prop_array[i][0]*all_bins_opt_prop_array[i][1])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    SSA_bulk_0W_18N_bin_den[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*all_bins_opt_prop_array[i][0])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    
    Q_scat[i] = all_bins_opt_prop_array[i][1] * all_bins_opt_prop_array[i][0]
    
    g_bulk_0W_18N_bin_num[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*Q_scat[i]*all_bins_opt_prop_array[i][2])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    g_bulk_0W_18N_bin_den[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*Q_scat[i])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    

In [ ]:
# Calculate values
Q_e_bulk_0W_18N_bulk = np.sum(Q_e_bulk_0W_18N_bin_num) / np.sum(Q_e_bulk_0W_18N_bin_den)
SSA_bulk_0W_18N_bulk = np.sum(SSA_bulk_0W_18N_bin_num) / np.sum(SSA_bulk_0W_18N_bin_den)
g_bulk_0W_18N_bulk = np.sum(g_bulk_0W_18N_bin_num) / np.sum(g_bulk_0W_18N_bin_den)

# Print the values
print("Q_e_bulk_0W_18N_bulk: ", Q_e_bulk_0W_18N_bulk)
print("SSA_bulk_0W_18N_bulk: ", SSA_bulk_0W_18N_bulk)
print("g_bulk_0W_18N_bulk: ", g_bulk_0W_18N_bulk)

In [ ]:
def create_if_not_exists(file_path):
    if not os.path.exists(file_path):
        with open(file_path, 'w') as file:
            # You can optionally write initial content to the file
            file.write("Wavelength\tQ_e_bulk_0W_18N_bulk\tSSA_bulk_0W_18N_bulk\tg_bulk_0W_18N_bulk\t" + '\n')
            #file.write(f"Q_e_bulk_0W_18N_bulk\t")
            #file.write(f"SSA_bulk_0W_18N_bulk\t")
            #file.write(f"g_bulk_0W_18N_bulk\t\n")
        print(f"File '{file_path}' created.")
    else:
        print(f"File '{file_path}' already exists.")

# Check if dust optical properties txt file already exists
file_path1 = 'RRTM_bulk_dust_optical_properties.txt'
file_path2 = 'RRTM_bulk_dust_optical_properties_{}.txt'.format(wavelength)
create_if_not_exists(file_path1)
create_if_not_exists(file_path2)

In [ ]:
# Assuming Q_e_bulk_0W_18N_bulk, SSA_bulk_0W_18N_bulk, g_bulk_0W_18N_bulk are numpy arrays

# Data to append
data_to_append = np.column_stack((wavelength, Q_e_bulk_0W_18N_bulk, SSA_bulk_0W_18N_bulk, g_bulk_0W_18N_bulk))

# Open the file in append mode and use np.savetxt to append data
with open('bulk_dust_optical_properties_{}.txt'.format(wavelength), 'a') as file:
    np.savetxt(file, data_to_append, fmt='%5.3f', delimiter='\t', header='', comments='')
    
# Open the file in append mode and use np.savetxt to append data
with open('bulk_dust_optical_properties.txt', 'a') as file:
    np.savetxt(file, data_to_append, fmt='%5.3f', delimiter='\t', header='', comments='')